###Working With FS CORS data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as g
import warnings
warnings.filterwarnings("ignore")

In [2]:
# List of Nutrients - Load all CSVs, set a consistent 'district' column,
# rename the last (value) column to the requested descriptive name, and drop duplicates by district
import glob
import os

csv_files = [
    'nutrients/Calcium.csv', 'nutrients/Foliate.csv', 'nutrients/Iron.csv', 'nutrients/Kilocaleries.csv',
    'nutrients/Proteins.csv', 'nutrients/Riboflavin.csv', 'nutrients/Thiamin.csv', 'nutrients/VitaminA.csv', 'nutrients/VitaminB12.csv',
    'nutrients/VitaminB6.csv', 'nutrients/VitaminC.csv', 'nutrients/Zinc.csv',
]

# Units (as provided) and pretty display names where needed
units = {
    'Calcium': 'mg',
    'Foliate': 'mcg',      # assuming Foliate -> Folate
    'Niacin': 'mg',        # not present in csv_files; kept for completeness
    'Proteins': 'mg',
    'Kilocaleries': 'kcal',
    'Iron': 'mg',
    'Riboflavin': 'mg',
    'Thiamin': 'mg',
    'VitaminA': 'mcg',
    'VitaminB12': 'mcg',
    'VitaminB6': 'mg',
    'VitaminC': 'mg',
    'Zinc': 'mg',
}

# Friendly display names (for nicer column titles)
display_name = {
    'Calcium': 'Calcium',
    'Foliate': 'Folate',
    'Proteins': 'Proteins',
    'Kilocaleries': 'Kilocaleries',
    'Iron': 'Iron',
    'Riboflavin': 'Riboflavin',
    'Thiamin': 'Thiamin',
    'VitaminA': 'Vitamin A',
    'VitaminB12': 'Vitamin B12',
    'VitaminB6': 'Vitamin B6',
    'VitaminC': 'Vitamin C',
    'Zinc': 'Zinc',
}

nutrient_dfs = {}
missing_files = []
for file in csv_files:
    if not os.path.exists(file):
        missing_files.append(file)
        continue
    df = pd.read_csv(file)
    # Ensure we have at least two columns (district and value)
    if df.shape[1] < 2:
        nutrient_dfs[file.replace('.csv','')] = df
        continue
    # Robustly identify the district column: prefer columns with 'name' or 'district' in the header, else use column index 1
    district_col = None
    for col in df.columns:
        lname = str(col).lower()
        if 'district' in lname or 'name' in lname:
            district_col = col
            break
    if district_col is None and len(df.columns) >= 2:
        district_col = df.columns[1]
    # Rename the district column to 'district'
    if district_col is not None:
        df = df.rename(columns={district_col: 'district'})
    # Build last-column display name based on filename base
    base = file.replace('.csv','')
    pretty = display_name.get(base, base)
    unit = units.get(base, '')
    # Compose the requested column name
    value_col_new = f'Average Consumption adequacy of {pretty} ({unit})' if unit else f'Average Consumption adequacy of {pretty}'
    # Rename the last column (whatever it currently is) to the composed name
    last_col = df.columns[-1]
    # If last_col is already 'district' (rare), try to find the numeric/value column instead
    if last_col == 'district' and df.shape[1] >= 2:
        last_col = df.columns[-1]  # fallback - keeps being last
    df = df.rename(columns={last_col: value_col_new})
    # Drop duplicates by district if the district column exists
    if 'district' in df.columns:
        df = df.drop_duplicates(subset=['district']).reset_index(drop=True)
    nutrient_dfs[base] = df

# Report missing files (if any) and show a sample head for verification
print('Missing files skipped:', missing_files)
if 'Calcium' in nutrient_dfs:
    nutrient_dfs['Calcium'].head()

Missing files skipped: []


In [3]:
#Obtaining dataframes
for df_name, df in nutrient_dfs.items():
    print(f"DataFrame for {df_name}:")
    print(df.head())

DataFrame for nutrients/Calcium:
   Category  district  Average Consumption adequacy of nutrients/Calcium
0         0      ABIM                                              66.99
1         2  ADJUMANI                                              22.91
2         3     AGAGO                                              64.41
3         5  ALEBTONG                                              63.12
4         7  AMOLATAR                                              45.78
DataFrame for nutrients/Foliate:
   Category  district  Average Consumption adequacy of nutrients/Foliate
0         0      ABIM                                              84.26
1         2  ADJUMANI                                              67.69
2         3     AGAGO                                              97.48
3         5  ALEBTONG                                              93.26
4         7  AMOLATAR                                              98.96
DataFrame for nutrients/Iron:
   Category  district  Avera

In [4]:
# Merge safe: reduce each dataframe to ['district', '<nutrient_value>'] where the value column is detected robustly
valid_dfs = {k: v for k, v in nutrient_dfs.items() if isinstance(v, pd.DataFrame) and 'district' in v.columns}
print('DataFrames found for merge:', list(valid_dfs.keys()))

if not valid_dfs:
    print('No valid DataFrames with a "district" column available to merge.')
else:
    cleaned = {}
    for name, df in valid_dfs.items():
        # columns except 'district'
        other_cols = [c for c in df.columns if c != 'district']
        if not other_cols:
            print(f'Skipping {name}: no value column found')
            continue
        # Prefer a column containing the 'Average Consumption adequacy' phrase
        val_col = None
        for c in other_cols:
            if 'Average Consumption adequacy' in str(c):
                val_col = c
                break
        # Fallback to numeric columns
        if val_col is None:
            numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
            numeric_cols = [c for c in numeric_cols if c != 'district']
            if numeric_cols:
                val_col = numeric_cols[0]
        # Final fallback: last non-district column
        if val_col is None:
            val_col = other_cols[-1]
        # Compose a clean, unique column name for this nutrient
        pretty = display_name.get(name, name)
        unit = units.get(name, '')
        new_col = f'Average Consumption adequacy of {pretty} ({unit})' if unit else f'Average Consumption adequacy of {pretty}'
        # Select and rename
        cleaned_df = df[['district', val_col]].copy()
        cleaned_df = cleaned_df.rename(columns={val_col: new_col})
        # Drop duplicates by district
        cleaned_df = cleaned_df.drop_duplicates(subset=['district']).reset_index(drop=True)
        cleaned[name] = cleaned_df
    # Now merge all cleaned dfs on 'district'
    from functools import reduce
    dfs_to_merge = list(cleaned.values())
    if not dfs_to_merge:
        print('No cleaned DataFrames to merge')
    else:
        merged_nutrients = reduce(lambda left, right: pd.merge(left, right, on='district', how='outer'), dfs_to_merge)
        merged_nutrients = merged_nutrients.sort_values('district').reset_index(drop=True)
        nutrient_dfs['merged_by_district'] = merged_nutrients
        print('Merged shape:', merged_nutrients.shape)
        merged_nutrients.head()

DataFrames found for merge: ['nutrients/Calcium', 'nutrients/Foliate', 'nutrients/Iron', 'nutrients/Kilocaleries', 'nutrients/Proteins', 'nutrients/Riboflavin', 'nutrients/Thiamin', 'nutrients/VitaminA', 'nutrients/VitaminB12', 'nutrients/VitaminB6', 'nutrients/VitaminC', 'nutrients/Zinc']


Merged shape: (122, 13)


In [5]:
merged_nutrients

,district,Average Consumption adequacy of nutrients/Calcium,Average Consumption adequacy of nutrients/Foliate,Average Consumption adequacy of nutrients/Iron,Average Consumption adequacy of nutrients/Kilocaleries,Average Consumption adequacy of nutrients/Proteins,Average Consumption adequacy of nutrients/Riboflavin,Average Consumption adequacy of nutrients/Thiamin,Average Consumption adequacy of nutrients/VitaminA,Average Consumption adequacy of nutrients/VitaminB12,Average Consumption adequacy of nutrients/VitaminB6,Average Consumption adequacy of nutrients/VitaminC,Average Consumption adequacy of nutrients/Zinc
0,ABIM,66.99,84.26,75.84,69.68,81.61,64.11,86.52,87.13,31.12,85.43,92.35,70.86
1,ADJUMANI,22.91,67.69,41.55,67.63,100.00,55.10,98.38,95.76,100.00,100.00,94.15,63.57
2,AGAGO,64.41,97.48,76.41,76.30,89.48,86.58,90.74,96.13,71.50,96.51,100.00,86.78
3,ALEBTONG,63.12,93.26,58.17,70.04,67.80,67.91,84.71,87.76,36.47,90.96,100.00,62.15
4,AMOLATAR,45.78,98.96,54.54,80.77,64.28,65.99,91.42,87.75,42.76,99.06,99.34,55.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,SSEMBABULE,69.75,98.65,70.40,88.81,89.47,86.86,92.35,85.88,45.22,100.00,100.00,78.24
118,TORORO,80.38,95.82,77.85,88.52,86.77,88.55,91.63,91.24,59.60,96.21,97.68,79.34
119,WAKISO,66.90,97.66,69.73,87.04,90.80,85.33,92.06,90.45,73.56,97.97,95.90,76.03
120,YUMBE,42.23,85.98,40.99,53.85,56.82,57.40,59.70,59.95,42.80,64.98,82.97,46.29


Average Consumption adequacy = (average in take of Nutrients / recommended intake of Nutrients )* 100

In [6]:
merged_nutrients.isna().sum()

district                                                  0
Average Consumption adequacy of nutrients/Calcium         0
Average Consumption adequacy of nutrients/Foliate         0
Average Consumption adequacy of nutrients/Iron            0
Average Consumption adequacy of nutrients/Kilocaleries    0
Average Consumption adequacy of nutrients/Proteins        0
Average Consumption adequacy of nutrients/Riboflavin      0
Average Consumption adequacy of nutrients/Thiamin         0
Average Consumption adequacy of nutrients/VitaminA        0
Average Consumption adequacy of nutrients/VitaminB12      0
Average Consumption adequacy of nutrients/VitaminB6       0
Average Consumption adequacy of nutrients/VitaminC        0
Average Consumption adequacy of nutrients/Zinc            0
dtype: int64

In [7]:
merged_nutrients.to_csv('merged_nutrients.csv', index=False)

In [8]:
merged_nutrients.duplicated().sum()

np.int64(0)

In [9]:
merged_nutrients.rename(columns={'Average Consumption adequacy of Kilocaleries (kcal)': 'Consumption adequacy of Kilocaleries (kcal)'}, inplace=True)

In [10]:
merged_nutrients

,district,Average Consumption adequacy of nutrients/Calcium,Average Consumption adequacy of nutrients/Foliate,Average Consumption adequacy of nutrients/Iron,Average Consumption adequacy of nutrients/Kilocaleries,Average Consumption adequacy of nutrients/Proteins,Average Consumption adequacy of nutrients/Riboflavin,Average Consumption adequacy of nutrients/Thiamin,Average Consumption adequacy of nutrients/VitaminA,Average Consumption adequacy of nutrients/VitaminB12,Average Consumption adequacy of nutrients/VitaminB6,Average Consumption adequacy of nutrients/VitaminC,Average Consumption adequacy of nutrients/Zinc
0,ABIM,66.99,84.26,75.84,69.68,81.61,64.11,86.52,87.13,31.12,85.43,92.35,70.86
1,ADJUMANI,22.91,67.69,41.55,67.63,100.00,55.10,98.38,95.76,100.00,100.00,94.15,63.57
2,AGAGO,64.41,97.48,76.41,76.30,89.48,86.58,90.74,96.13,71.50,96.51,100.00,86.78
3,ALEBTONG,63.12,93.26,58.17,70.04,67.80,67.91,84.71,87.76,36.47,90.96,100.00,62.15
4,AMOLATAR,45.78,98.96,54.54,80.77,64.28,65.99,91.42,87.75,42.76,99.06,99.34,55.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,SSEMBABULE,69.75,98.65,70.40,88.81,89.47,86.86,92.35,85.88,45.22,100.00,100.00,78.24
118,TORORO,80.38,95.82,77.85,88.52,86.77,88.55,91.63,91.24,59.60,96.21,97.68,79.34
119,WAKISO,66.90,97.66,69.73,87.04,90.80,85.33,92.06,90.45,73.56,97.97,95.90,76.03
120,YUMBE,42.23,85.98,40.99,53.85,56.82,57.40,59.70,59.95,42.80,64.98,82.97,46.29


In [11]:
merged_nutrients.columns

Index(['district', 'Average Consumption adequacy of nutrients/Calcium',
       'Average Consumption adequacy of nutrients/Foliate',
       'Average Consumption adequacy of nutrients/Iron',
       'Average Consumption adequacy of nutrients/Kilocaleries',
       'Average Consumption adequacy of nutrients/Proteins',
       'Average Consumption adequacy of nutrients/Riboflavin',
       'Average Consumption adequacy of nutrients/Thiamin',
       'Average Consumption adequacy of nutrients/VitaminA',
       'Average Consumption adequacy of nutrients/VitaminB12',
       'Average Consumption adequacy of nutrients/VitaminB6',
       'Average Consumption adequacy of nutrients/VitaminC',
       'Average Consumption adequacy of nutrients/Zinc'],
      dtype='object')

In [12]:
merged_nutrients.describe()

,Average Consumption adequacy of nutrients/Calcium,Average Consumption adequacy of nutrients/Foliate,Average Consumption adequacy of nutrients/Iron,Average Consumption adequacy of nutrients/Kilocaleries,Average Consumption adequacy of nutrients/Proteins,Average Consumption adequacy of nutrients/Riboflavin,Average Consumption adequacy of nutrients/Thiamin,Average Consumption adequacy of nutrients/VitaminA,Average Consumption adequacy of nutrients/VitaminB12,Average Consumption adequacy of nutrients/VitaminB6,Average Consumption adequacy of nutrients/VitaminC,Average Consumption adequacy of nutrients/Zinc
count,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000
mean,63.692459,95.475902,67.644918,80.608443,86.999754,80.492213,88.938443,87.189918,62.075164,94.095820,97.696967,72.828525
std,14.202116,6.265266,13.290775,12.813223,10.904073,12.476063,10.005677,11.393070,22.607859,8.840154,8.823652,12.111087
min,7.870000,67.690000,24.890000,43.440000,55.900000,36.800000,56.990000,17.760000,0.000000,59.620000,9.050000,32.600000
25%,56.895000,93.935000,58.977500,74.057500,81.770000,73.487500,84.790000,83.917500,47.065000,91.180000,98.020000,66.120000
50%,65.750000,97.780000,67.155000,83.445000,88.650000,82.875000,91.085000,88.085000,62.145000,98.490000,100.000000,73.520000
75%,71.752500,100.000000,76.860000,89.372500,96.155000,88.572500,96.245000,94.495000,79.197500,100.000000,100.000000,81.525000
max,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


### Index Vulneablilty Analysis


In [26]:
from functools import reduce
import pandas as pd

files = [
    ("Index datasets/composite-vulnerability.csv", "Composite Vulnerability Index"),
    ("Index datasets/health-system-vulnerabil.csv", "Health System Vulnerability Index"),
    ("Index datasets/mean-adequacy-ratio-inde.csv", "Mean Adequacy Ratio Index"),
    ("Index datasets/per-capita-food-consumpt.csv", "Per Capita Food Consumption Index"),
    ("Index datasets/vulnerability-to-climate.csv", "Vulnerability to Climate Change Index"),
]

dfs = []
for path, name in files:
    df = pd.read_csv(path)
    df.columns = ["Category", name]
    dfs.append(df)

merged_index = reduce(lambda left, right: pd.merge(left, right, on="Category", how="outer"), dfs)
merged_index = merged_index.rename(columns={'Category': 'Region'})
merged_index.head()
merged_index.to_csv("merged_index.csv", index=False)

In [27]:
merged_index.head()

,Region,Composite Vulnerability Index,Health System Vulnerability Index,Mean Adequacy Ratio Index,Per Capita Food Consumption Index,Vulnerability to Climate Change Index
0,Acholi,0.8680,0.9266,1.0000,0.1938,0.7391
1,Ankole,0.4817,0.5628,0.2435,0.7355,0.8560
2,Buganda North,0.4963,0.4324,0.3440,0.4525,0.6615
3,Buganda South,0.4064,0.3024,0.2186,0.4134,0.5181
4,Bukedi,0.4659,0.4699,0.3210,0.4226,0.4953


### Communicating  Composite Vulnerability   
Composite Vulnerability Index – An overall combined measure of vulnerability across multiple dimensions (health, food, climate, etc.). A higher value suggests higher vulnerability.

In [34]:
composite=merged_index[['Region','Composite Vulnerability Index']].sort_values(by='Composite Vulnerability Index', ascending=False).drop_duplicates(subset=['Region']).reset_index(drop=True)
composite.head()

,Region,Composite Vulnerability Index
0,Acholi,0.8680
1,Lango,0.7666
2,Busoga,0.6745
3,Karamoja,0.6228
4,Teso,0.5854


In [35]:
# Bar Composite Vulnerability by Category
fig = px.bar(
    composite,
    x="Region",  # Regions (Acholi, Buganda, etc.)
    y="Composite Vulnerability Index",  # Value to plot
    title="Composite Vulnerability Index by Region",
    color="Composite Vulnerability Index",  # Color scale based on value
    #color_continuous_scale="Reds"
)

# Rotate x-axis labels for readability
fig.update_layout(xaxis_tickangle=-45,
                  coloraxis_colorbar=dict(title="Vulnerability Index"))


### Composite Vulnerability Index by Region  

The **Composite Vulnerability Index (CVI)** is the average of four dimensions:  
- **Health System Vulnerability**  
- **Mean Adequacy Ratio (nutrition adequacy)**  
- **Per Capita Food Consumption**  
- **Vulnerability to Climate Change**  

This provides a single measure (0 = low, 1 = high) to compare regions.  

> **Insight:** Acholi, Lango, and Busoga are the most vulnerable regions, while Kampala and Elgon are the least.  
> This suggests interventions should target the high-risk regions first to strengthen resilience.  


### Healthy System Vulnerability


In [36]:
health=merged_index[['Region','Health System Vulnerability Index']].sort_values(by='Health System Vulnerability Index', ascending=False).drop_duplicates(subset=['Region']).reset_index(drop=True)
health.head()

,Region,Health System Vulnerability Index
0,Lango,1.0000
1,Acholi,0.9266
2,Busoga,0.8489
3,Teso,0.8167
4,Toro,0.5958


In [37]:
# Plot the health vulnerability dataframe.
# Use the numeric 'Health System Vulnerability Index' as the marker size
# (the previous size="pop" raised an error because 'pop' doesn't exist in `health`)
fig = px.scatter(
	health,
	x="Health System Vulnerability Index",
	y="Region",
	size="Health System Vulnerability Index",
	color="Health System Vulnerability Index",
	title="Health System Vulnerability Index by Region",
	labels={"Category": "Region", "Health System Vulnerability Index": "Health System Vulnerability Index"}
)
# Order categories by value for clearer display
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

### Health System Vulnerability Index

The **Health System Vulnerability Index** measures a region’s susceptibility to health system challenges.  
**Scale:** 0 = low vulnerability, 1 = high vulnerability.

> **Insight:**  
> **Lango** is the most vulnerable region, while **Kampala** is the least.  
> Prioritize interventions in high-risk areas like **Lango** to improve resilience.


### Food Security

In [38]:
Food_security=merged_index[['Region','Mean Adequacy Ratio Index','Per Capita Food Consumption Index']].drop_duplicates(subset=['Region']).reset_index(drop=True)
Food_security.head()

,Region,Mean Adequacy Ratio Index,Per Capita Food Consumption Index
0,Acholi,1.0000,0.1938
1,Ankole,0.2435,0.7355
2,Buganda North,0.3440,0.4525
3,Buganda South,0.2186,0.4134
4,Bukedi,0.3210,0.4226


In [39]:
# Create scatter plot using the Food_security dataframe and correct column names
fig = px.scatter(
    Food_security,
    x='Per Capita Food Consumption Index',
    y='Mean Adequacy Ratio Index',
    color='Region',
    hover_data=['Region'],
    labels={
        'Per Capita Food Consumption Index': 'Per Capita Food Consumption (index)',
        'Mean Adequacy Ratio Index': 'Mean Adequacy Ratio (index)',
        'Category': 'Region'
    },
    title='Nutrition Adequacy vs Food Consumption by Region'
)
fig.update_traces(marker=dict(size=10, opacity=0.8, line=dict(width=2, color='DarkSlateGrey')))
fig.show()


### Nutrition Adequacy vs Food Consumption by Region  

This chart compares **per capita food consumption** with the **mean adequacy ratio of nutrients** across regions.  
- Regions higher on the chart have **better nutrient adequacy**, even if food consumption is low.  
- Regions further right show **higher food consumption**, though this does not always guarantee adequate nutrition.  

> **Insight:**  
> Some regions consume more food but still have low nutrient adequacy, highlighting the need for **diverse diets and nutrition-focused interventions**.  


## Climate Change Vulnerability index

In [40]:
composite=merged_index[['Region','Vulnerability to Climate Change Index']].sort_values(by='Vulnerability to Climate Change Index', ascending=False).drop_duplicates(subset=['Region']).reset_index(drop=True)
composite.head()

,Region,Vulnerability to Climate Change Index
0,Kigezi,1.0000
1,Busoga,0.9336
2,Ankole,0.8560
3,Karamoja,0.7933
4,West Nile,0.7678


In [41]:
# Bar Composite Vulnerability by Category
fig = px.bar(
    composite,
    x="Region",  # Regions (Acholi, Buganda, etc.)
    y="Vulnerability to Climate Change Index",  # Value to plot
    title="Vulnerability to Climate Change Index by Region",
    color="Vulnerability to Climate Change Index",  # Color scale based on value
    #color_continuous_scale="Reds"
)

# Rotate x-axis labels for readability
fig.update_layout(xaxis_tickangle=-45,
                  coloraxis_colorbar=dict(title="Vulnerability Index"))


### Vulnerability to Climate Change Index by Region  

This chart shows how different regions are exposed to the risks of **climate change**, with values ranging from 0 (low vulnerability) to 1 (high vulnerability).  

- **Kigezi, Busoga, and Ankole** show the **highest vulnerability**, meaning they face the greatest climate risks.  
- **Kampala and Elgon** are the **least vulnerable**, but resilience strategies remain important.  

> **Insight:**  
> Regions like **Kigezi and Busoga** need urgent climate adaptation measures, while lower-risk regions should focus on sustaining their resilience.  
